# ProjectQ Compiler Tutorial


The aim of this short tutorial is to give a first introduction to the ProjectQ compiler. In particular, we will show how to specify the gate set to which the compiler should translate a quantum program. A more extended tutorial will follow soon. Please check out our [ProjectQ paper](http://arxiv.org/abs/1612.08091) for an introduction to the basic concepts behind our compiler.

## The default compiler

To compile a quantum program, we begin by creating a compiler called `MainEngine` and specify the backend for which the compiler should translate the program. For the purpose of this tutorial, we will use a `CommandPrinter` as a backend to display the compiled algorithm. It works the same for all other backends such as, e.g., the simulator or an interface to real hardware.

Let's write a small program:

In [1]:
import projectq
from projectq.backends import CommandPrinter
from projectq.meta import Control
from projectq.ops import All, CNOT, Measure, QFT, QubitOperator, Rx, TimeEvolution, X

# create the compiler and specify the backend:
eng = projectq.MainEngine(backend=CommandPrinter(accept_input=False))

def my_quantum_program(eng):
    qubit = eng.allocate_qubit()
    qureg = eng.allocate_qureg(3)
    with Control(eng, qubit):
        hamiltonian = 0.5 * QubitOperator("X0 Y1 Z2")
        TimeEvolution(0.1, hamiltonian) | qureg
    QFT | qureg
    Rx(0.1) | qubit
    CNOT | (qubit, qureg[0])
    All(Measure) | qureg
    Measure | qubit
    eng.flush()
my_quantum_program(eng)

Allocate | Qureg[1]
Allocate | Qureg[0]
Allocate | Qureg[2]
Allocate | Qureg[3]
Cexp(-0.1j * (0.5 X0 Y1 Z2)) | ( Qureg[0], Qureg[1-3] )
QFT | Qureg[1-3]
Rx(0.1) | Qureg[0]
CX | ( Qureg[0], Qureg[1] )
Measure | Qureg[1]
Measure | Qureg[2]
Measure | Qureg[3]
Measure | Qureg[0]
Deallocate | Qureg[0]
Deallocate | Qureg[3]
Deallocate | Qureg[2]
Deallocate | Qureg[1]


In the above example, the compiler did nothing because the default compiler (when `MainEngine` is called without a specific `engine_list` parameter) translates the individual gates to the gate set supported by the backend. In our case, the backend is a `CommandPrinter` which supports any type of gate.

We can check what happens when the backend is a `Simulator` by inserting a `CommandPrinter` as a last compiler engine before the backend so that every command is printed before it gets sent to the Simulator: 

In [2]:
from projectq.backends import Simulator

# Use the default compiler engines with a CommandPrinter in the end:
engines2 = projectq.default_engines() + [CommandPrinter()]

eng2 = projectq.MainEngine(backend=Simulator(), engine_list=engines2)
my_quantum_program(eng2)

Allocate | Qureg[1]
Allocate | Qureg[0]
Allocate | Qureg[2]
Allocate | Qureg[3]
Cexp(-0.1j * (0.5 X0 Y1 Z2)) | ( Qureg[0], Qureg[1-3] )
H | Qureg[3]
CR(1.5707963267948966) | ( Qureg[2], Qureg[3] )
CR(0.7853981633974483) | ( Qureg[1], Qureg[3] )
H | Qureg[2]
CR(1.5707963267948966) | ( Qureg[1], Qureg[2] )
H | Qureg[1]
Rx(0.1) | Qureg[0]
CX | ( Qureg[0], Qureg[1] )
Measure | Qureg[1]
Measure | Qureg[2]
Measure | Qureg[3]
Measure | Qureg[0]
Deallocate | Qureg[0]
Deallocate | Qureg[3]
Deallocate | Qureg[2]
Deallocate | Qureg[1]


As one can see, in this case the compiler had to do a little work because the Simulator does not support a QFT gate. Therefore, it automatically replaces the QFT gate by a sequence of lower-level gates.

## Specifying a particular gate set

In this short example, we want to explore how to specify a particular gate set, which may be even more restrictive than what the backend naturally supports. This is useful, e.g., to obtain resource estimates for running a given program on actual quantum hardware which, in this example, can only perform CNOT and single qubit gates. All one has to do is insert an `InstructionFilter` into the `engine_list`:

In [3]:
from projectq.cengines import InstructionFilter
from projectq.ops import ClassicalInstructionGate

# Write a function which, given a Command object, returns whether the command is supported:
def is_supported(eng, cmd):
    if isinstance(cmd.gate, ClassicalInstructionGate):
        # This is required to allow Measure, Allocate, Deallocate, Flush
        return True
    elif isinstance(cmd.gate, X.__class__) and len(cmd.control_qubits) == 1:
        # Allows a CNOT gate which is an X gate with one control qubit
        return True
    elif (len(cmd.control_qubits) == 0 and 
          len(cmd.qubits) == 1 and
          len(cmd.qubits[0]) == 1):
        # Gate which has no control qubits, applied to 1 qureg consisting of 1 qubit
        return True
    else:
        return False

supported_gate_set_filter = InstructionFilter(is_supported)

# Append the instruction filter to the list of compiler engines:
engines3 = projectq.default_engines() + [supported_gate_set_filter]

eng3 = projectq.MainEngine(backend=CommandPrinter(accept_input=False), engine_list=engines3)
my_quantum_program(eng3)

Allocate | Qureg[3]
Allocate | Qureg[2]
Rx(1.5707963267948966) | Qureg[2]
Allocate | Qureg[1]
H | Qureg[1]
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[2], Qureg[3] )
Rz(0.05) | Qureg[3]
Allocate | Qureg[0]
CX | ( Qureg[0], Qureg[3] )
Rz(12.516370614359172) | Qureg[3]
CX | ( Qureg[0], Qureg[3] )
CX | ( Qureg[2], Qureg[3] )
CX | ( Qureg[1], Qureg[2] )
H | Qureg[1]
R(0.39269908169872414) | Qureg[1]
H | Qureg[3]
Rz(0.7853981633974483) | Qureg[3]
Rx(10.995574287564276) | Qureg[2]
R(0.7853981633974483) | Qureg[2]
CX | ( Qureg[2], Qureg[3] )
Rz(11.780972450961723) | Qureg[3]
CX | ( Qureg[2], Qureg[3] )
Rz(0.39269908169872414) | Qureg[3]
CX | ( Qureg[1], Qureg[3] )
Rz(12.173671532660448) | Qureg[3]
CX | ( Qureg[1], Qureg[3] )
R(0.7853981633974483) | Qureg[1]
H | Qureg[2]
Rz(0.7853981633974483) | Qureg[2]
CX | ( Qureg[1], Qureg[2] )
Rz(11.780972450961723) | Qureg[2]
CX | ( Qureg[1], Qureg[2] )
H | Qureg[1]
Rx(0.1) | Qureg[0]
CX | ( Qureg[0], Qureg[1] )
Measure | Qureg[1]
Measure | Qureg[2]
Measure

As we can see, the compiler now needs to do a little more work. In some cases, the compiler does not know how to translate a command according to the specified gate set and it will throw a `NoGateDecompositionError`. This means one needs to implement a rule specifying how to decompose said command. See projectq/setups/decompositions for a few examples. This will be explained in a more extended tutorial.